In [1]:
import mysql.connector
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import os
from dotenv import load_dotenv

In [2]:
pd.set_option("display.max_columns", None)

Load data from MySQL

In [3]:
load_dotenv()

# Event data
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    database=os.getenv("DB_NAME"),
    password=os.getenv("DB_PASSWORD")
    )

cursor = conn.cursor()

query = ("SELECT * FROM events")

cursor.execute(query)

columns = [desc[0] for desc in cursor.description]

rows = cursor.fetchall()

df_events = pd.DataFrame(rows, columns=columns)

cursor.close()
conn.close()

In [4]:
# Stats data
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    database=os.getenv("DB_NAME"),
    password=os.getenv("DB_PASSWORD")
    )

cursor = conn.cursor()

query = ("SELECT * FROM stats")

cursor.execute(query)

columns = [desc[0] for desc in cursor.description]

rows = cursor.fetchall()

df_stats = pd.DataFrame(rows, columns=columns)

cursor.close()
conn.close()

Preview datasets

In [9]:
df_events.tail()


,id,event_date,event_name,fighter_blue,fighter_red,knockdowns_blue,knockdowns_red,sig_attempts_blue,sig_attempts_red,sig_strikes_blue,sig_strikes_red,total_strikes_attempts_blue,total_strikes_attempts_red,total_strikes_blue,total_strikes_red,sub_attempts_blue,sub_attempts_red,takedowns_blue,takedowns_red,takedown_attempts_blue,takedown_attempts_red,control_time_blue,control_time_red,round,time,weight_class,win_method,winner
8298,8299,2025-08-23,UFC Fight Night: Walker vs. Zhang,Kevin Borjas,Sumudaerji,0,0,44,129,16,73,51,150,23,91,0,0,0,0,5,5,209,17,3,300,Flyweight,Decision - Unanimous,Sumudaerji
8299,8300,2025-08-23,UFC Fight Night: Walker vs. Zhang,Zhang Mingyang,Johnny Walker,0,1,47,67,20,50,59,70,28,51,0,0,0,0,0,2,81,80,2,157,Light Heavyweight,KO/TKO,Johnny Walker
8300,8301,2025-08-23,UFC Fight Night: Walker vs. Zhang,Aljamain Sterling,Brian Ortega,0,0,227,174,124,55,262,225,153,100,0,0,3,0,6,2,235,8,5,300,Catch Weight,Decision - Unanimous,Aljamain Sterling
8301,8302,2025-08-23,UFC Fight Night: Walker vs. Zhang,Diyar Nurgozhay,Uran Satybaldiev,0,1,11,9,7,2,19,10,13,3,0,2,1,0,1,0,18,70,1,165,Light Heavyweight,Submission,Uran Satybaldiev
8302,8303,2025-08-23,UFC Fight Night: Walker vs. Zhang,Westin Wilson,Yizha,0,2,2,18,0,13,2,23,0,17,0,0,0,0,0,0,0,6,1,37,Featherweight,KO/TKO,Yizha


In [8]:
df_stats.tail()

,id,name,nickname,division,record,status,place_of_birth,trains_at,fighting_style,octagon_debut,age,height,weight,reach,leg_reach,wins,losses,draws,wins_by_knockout,first_round_finishes,win_by_dec,win_by_sub,sig_strikes_landed,sig_strikes_attempted,takedowns_landed,takedowns_attempted,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,sig_strikes_defense,takedown_defense,knockdown_avg,fight_time_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
2995,2996,Zarah Fairn,"""Infinite""",Women's Bantamweight Division,6-5-0 (W-L-D),Active,"Paris, France",Team Figueiredo,MMA,"Oct. 5, 2019",39.0,68.0,146.0,72.0,42.0,6,5,0,4.0,4.0,2.0,0.0,171.0,543.0,NaN,2.0,5.0,7.0,0.00,0.00,46.0,60.0,0.00,09:24,160.0,11.0,0.0,128.0,26.0,17.0
2996,2997,Zu Anyanwu,"""8th Wonder""",Heavyweight Division,14-5-0 (W-L-D),Not Fighting,"Trenton, United States",None,None,"Jul. 11, 2017",41.0,73.0,263.5,77.0,41.0,14,5,0,NaN,NaN,4.0,2.0,57.0,133.0,NaN,1.0,2.0,2.0,0.65,0.00,69.0,NaN,0.65,11:32,48.0,3.0,6.0,46.0,11.0,0.0
2997,2998,Zygimantas Ramaska,None,Featherweight Division,9-3-0 (W-L-D),Not Fighting,Lithuania,Ukmergė Judo Club,Judo,"Aug. 24, 2024",28.0,71.0,145.5,73.0,NaN,9,3,0,5.0,5.0,0.0,4.0,6.0,10.0,NaN,NaN,1.0,2.0,0.00,2.41,30.0,33.0,0.00,06:14,2.0,4.0,0.0,5.0,1.0,0.0
2998,2999,Zviad Lazishvili,None,Bantamweight Division,13-1-0 (W-L-D),Not Fighting,"Kobuleti, Georgia",Kaizen MMA,Freestyle,"Oct. 23, 2021",31.0,66.0,135.0,69.0,37.0,13,1,0,NaN,NaN,4.0,9.0,63.0,160.0,NaN,6.0,4.0,6.0,0.00,0.00,51.0,NaN,0.00,15:00,60.0,3.0,0.0,25.0,17.0,21.0
2999,3000,Zak Cummings,None,Light Heavyweight Division,25-7-0 (W-L-D),Not Fighting,"Irving, United States",Glory MMA & Fitness,Jiu-Jitsu,"Aug. 29, 2013",38.0,72.0,206.0,75.0,40.0,25,7,0,5.0,NaN,6.0,12.0,500.0,1394.0,7.0,25.0,3.0,3.0,0.62,0.62,55.0,67.0,0.71,11:17,403.0,59.0,38.0,302.0,93.0,105.0


Review Stats

In [7]:
df_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 40 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               3000 non-null   int64  
 1   name                             3000 non-null   object 
 2   nickname                         1784 non-null   object 
 3   division                         2884 non-null   object 
 4   record                           2958 non-null   object 
 5   status                           2968 non-null   object 
 6   place_of_birth                   2827 non-null   object 
 7   trains_at                        964 non-null    object 
 8   fighting_style                   997 non-null    object 
 9   octagon_debut                    3000 non-null   object 
 10  age                              2735 non-null   float64
 11  height                           2720 non-null   float64
 12  weight              

In [10]:
df_stats.describe()

,id,age,height,weight,reach,leg_reach,wins,losses,draws,wins_by_knockout,first_round_finishes,win_by_dec,win_by_sub,sig_strikes_landed,sig_strikes_attempted,takedowns_landed,takedowns_attempted,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,sig_strikes_defense,takedown_defense,knockdown_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
count,3000.000000,2735.000000,2720.000000,2893.000000,1893.000000,1614.000000,3000.000000,3000.000000,3000.000000,1184.000000,848.000000,2947.000000,2947.000000,2700.000000,2700.000000,894.000000,2425.000000,2709.000000,2709.000000,2709.000000,2709.000000,2697.000000,2282.000000,2709.000000,2947.000000,2947.000000,2947.000000,2947.000000,2947.000000,2947.000000
mean,1500.500000,36.228154,65.480147,157.430245,71.615689,39.919455,11.485667,4.400000,0.166667,6.306588,6.307783,2.480489,2.377672,248.089259,554.927407,7.291946,21.361237,3.153193,3.881506,1.475799,0.611694,52.044865,61.230938,0.292503,164.582287,31.418731,31.294876,143.343400,47.048185,36.904309
std,866.169729,7.099194,17.819406,51.661631,4.303875,2.620799,8.716141,3.747979,0.586234,4.050162,3.780998,3.466924,3.552184,313.206002,694.562397,10.352186,28.260367,1.684430,2.480274,1.700987,1.206330,10.461133,21.188051,0.618165,238.320420,48.156931,52.339318,199.138028,68.379187,56.836150
min,1.000000,19.000000,0.000000,0.000000,58.500000,32.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,4.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,750.750000,31.000000,67.000000,136.000000,69.000000,38.000000,6.000000,2.000000,0.000000,3.000000,4.000000,0.000000,0.000000,45.000000,109.000000,2.000000,4.000000,2.000000,3.000000,0.220000,0.000000,46.000000,48.000000,0.000000,17.000000,3.000000,0.000000,16.000000,5.000000,3.000000
50%,1500.500000,35.000000,70.000000,156.000000,72.000000,40.000000,11.000000,4.000000,0.000000,6.000000,6.000000,1.000000,1.000000,130.000000,285.500000,4.000000,11.000000,3.000000,3.000000,1.020000,0.250000,53.000000,62.000000,0.000000,74.000000,13.000000,11.000000,67.000000,21.000000,16.000000
75%,2250.250000,40.000000,73.000000,185.000000,74.500000,41.500000,16.000000,6.000000,0.000000,8.000000,8.000000,4.000000,4.000000,328.000000,740.250000,8.000000,26.000000,4.000000,5.000000,2.130000,0.820000,59.000000,75.000000,0.410000,220.000000,39.000000,39.000000,189.500000,62.000000,46.000000
max,3000.000000,80.000000,84.000000,415.000000,84.500000,74.000000,88.000000,24.000000,10.000000,28.000000,26.000000,36.000000,47.000000,3655.000000,7602.000000,90.000000,281.000000,14.000000,53.000000,24.110000,21.950000,100.000000,100.000000,12.330000,3222.000000,503.000000,523.000000,2363.000000,893.000000,508.000000


In [11]:
df_stats.isnull().sum()

id                                    0
name                                  0
nickname                           1216
division                            116
record                               42
status                               32
place_of_birth                      173
trains_at                          2036
fighting_style                     2003
octagon_debut                         0
age                                 265
height                              280
weight                              107
reach                              1107
leg_reach                          1386
wins                                  0
losses                                0
draws                                 0
wins_by_knockout                   1816
first_round_finishes               2152
win_by_dec                           53
win_by_sub                           53
sig_strikes_landed                  300
sig_strikes_attempted               300
takedowns_landed                   2106


Find Duplicates Names and Drop

In [12]:
df_stats[df_stats.duplicated('name', keep=False) == True]

,id,name,nickname,division,record,status,place_of_birth,trains_at,fighting_style,octagon_debut,age,height,weight,reach,leg_reach,wins,losses,draws,wins_by_knockout,first_round_finishes,win_by_dec,win_by_sub,sig_strikes_landed,sig_strikes_attempted,takedowns_landed,takedowns_attempted,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,sig_strikes_defense,takedown_defense,knockdown_avg,fight_time_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
386,387,Bruno Silva,"""Bulldog""",Flyweight Division,14-7-2 (W-L-D),Active,"Piracicaba, Brazil",American Top Team,Grappler,"Sep. 7, 2019",35.0,64.0,125.0,65.0,35.0,14,7,2,6.0,4.0,4.0,4.0,353.0,713.0,2.0,46.0,4.0,5.0,1.79,0.16,50.0,60.0,0.98,10:15,287.0,24.0,42.0,196.0,80.0,77.0
387,388,Bruno Silva,"""Bulldog""",Flyweight Division,14-7-2 (W-L-D),Active,"Piracicaba, Brazil",American Top Team,Grappler,"Sep. 7, 2019",35.0,64.0,125.0,65.0,35.0,14,7,2,6.0,4.0,4.0,4.0,353.0,713.0,2.0,46.0,4.0,5.0,1.79,0.16,50.0,60.0,0.98,10:15,287.0,24.0,42.0,196.0,80.0,77.0
453,454,Casey Kenney,None,Bantamweight Division,16-4-1 (W-L-D),Active,"Portland, United States",None,MMA,"Jul. 18, 2017",32.0,67.0,136.0,70.0,36.0,16,4,1,2.0,NaN,9.0,5.0,670.0,1570.0,2.0,28.0,5.0,5.0,1.09,0.43,58.0,61.0,0.00,13:48,598.0,54.0,18.0,361.0,168.0,141.0
454,455,Casey Kenney,None,Bantamweight Division,16-4-1 (W-L-D),Active,"Portland, United States",None,MMA,"Jul. 18, 2017",32.0,67.0,136.0,70.0,36.0,16,4,1,2.0,NaN,9.0,5.0,670.0,1570.0,2.0,28.0,5.0,5.0,1.09,0.43,58.0,61.0,0.00,13:48,598.0,54.0,18.0,361.0,168.0,141.0
1379,1380,Joey Gomez,"""KO King""",Bantamweight Division,6-2-0 (W-L-D),Not Fighting,"Fairfax, United States",Team Link Hooksett,None,"Jan. 18, 2016",36.0,68.0,135.0,73.0,40.0,6,2,0,NaN,NaN,0.0,0.0,59.0,207.0,NaN,1.0,2.0,4.0,0.62,0.00,55.0,50.0,0.62,12:07,47.0,12.0,0.0,36.0,9.0,14.0
1380,1381,Joey Gomez,"""KO King""",Bantamweight Division,6-2-0 (W-L-D),Not Fighting,"Fairfax, United States",Team Link Hooksett,None,"Jan. 18, 2016",36.0,68.0,135.0,73.0,40.0,6,2,0,NaN,NaN,0.0,0.0,59.0,207.0,NaN,1.0,2.0,4.0,0.62,0.00,55.0,50.0,0.62,12:07,47.0,12.0,0.0,36.0,9.0,14.0
2378,2379,Richie Vaculik,"""Vas""",Flyweight Division,10-5-0 (W-L-D),Not Fighting,"Sydney, Australia",None,None,"Dec. 7, 2013",40.0,66.0,125.0,69.0,41.0,10,5,0,NaN,NaN,1.0,6.0,166.0,355.0,NaN,20.0,4.0,4.0,3.00,1.00,57.0,32.0,0.00,11:15,132.0,23.0,11.0,107.0,31.0,28.0
2379,2380,Richie Vaculik,"""Vas""",Flyweight Division,10-5-0 (W-L-D),Not Fighting,"Sydney, Australia",None,None,"Dec. 7, 2013",40.0,66.0,125.0,69.0,41.0,10,5,0,NaN,NaN,1.0,6.0,166.0,355.0,NaN,20.0,4.0,4.0,3.00,1.00,57.0,32.0,0.00,11:15,132.0,23.0,11.0,107.0,31.0,28.0


In [13]:
df_stats.drop_duplicates(subset=['name'], inplace=True)

Find missing values

In [15]:
missing_df = pd.DataFrame({
    'missing_count': df_stats.isnull().sum(),
    'missing_precent': round((df_stats.isnull().sum() / len(df_stats)) * 100, 2)
}).sort_values(by='missing_precent', ascending=False)

missing_df

,missing_count,missing_precent
first_round_finishes,2149,71.73
takedowns_landed,2104,70.23
trains_at,2034,67.89
fighting_style,2001,66.79
wins_by_knockout,1814,60.55
leg_reach,1386,46.26
nickname,1215,40.55
reach,1107,36.95
takedown_defense,718,23.97
takedowns_attempted,575,19.19


Remove null values with these columns. They are the new fighters that still do not have stats and a few very old fighters with also no stats

In [16]:
df_stats[df_stats[['sig_strikes_standing', 'sig_strikes_clinch', 'sig_strikes_ground', 'head_target', 'body_target', 'leg_target']].isnull().all(axis=1)]

,id,name,nickname,division,record,status,place_of_birth,trains_at,fighting_style,octagon_debut,age,height,weight,reach,leg_reach,wins,losses,draws,wins_by_knockout,first_round_finishes,win_by_dec,win_by_sub,sig_strikes_landed,sig_strikes_attempted,takedowns_landed,takedowns_attempted,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,sig_strikes_defense,takedown_defense,knockdown_avg,fight_time_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
54,55,Alberta Cerra,None,None,None,Not Fighting,United States,None,None,"Aug. 24, 2025",NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
137,138,Alvaro Ivan Lopez Rodrigues,None,None,None,Not Fighting,Mexico,None,None,"Aug. 24, 2025",NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
150,151,Amir Aliakbari,None,Heavyweight Division,0-0-0 (W-L-D),Not Fighting,None,None,None,"Aug. 24, 2025",39.0,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
244,245,Artem Frolov,None,Middleweight Division,0-0-0 (W-L-D),Not Fighting,None,None,None,"Aug. 24, 2025",31.0,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
248,249,Asikeerbai Jinensibieke,None,None,None,Active,"Xinjiang, China",None,None,"Jun. 10, 2022",34.0,NaN,155.0,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
314,315,Bilyal Makhov,None,None,None,Retired,"USSR, Russia",None,None,"Aug. 24, 2025",35.0,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
448,449,Carrese Archer,"""One Punch""",None,None,Not Fighting,None,None,None,"Aug. 24, 2025",33.0,72.0,184.0,73.0,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
493,494,Chi Lewis-Parry,None,Heavyweight Division,0-0-0 (W-L-D),Not Fighting,"Hitchin, United Kingdom",None,None,"Aug. 24, 2025",39.0,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
562,563,Cody Belisle,"""Bodacious""",None,None,Active,None,None,None,"Aug. 24, 2025",30.0,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
590,591,Cory Corbin,"""Hamma""",None,None,Active,None,None,None,"Aug. 24, 2025",36.0,NaN,254.0,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_stats = df_stats[~df_stats[['sig_strikes_standing', 'sig_strikes_clinch', 'sig_strikes_ground', 'head_target', 'body_target', 'leg_target']].isnull().all(axis=1)]

Drop irrelevant columns that also have a high null percentage

In [19]:
df_stats.drop(columns=['id', 'nickname', 'division', 'record', 'status', 'place_of_birth', 'trains_at', 'fighting_style'], inplace=True)

Convert octagon_debut to datetime

In [20]:
df_stats['octagon_debut'] = pd.to_datetime(df_stats['octagon_debut'], format='%b. %d, %Y')
df_stats

,name,octagon_debut,age,height,weight,reach,leg_reach,wins,losses,draws,wins_by_knockout,first_round_finishes,win_by_dec,win_by_sub,sig_strikes_landed,sig_strikes_attempted,takedowns_landed,takedowns_attempted,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,sig_strikes_defense,takedown_defense,knockdown_avg,fight_time_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
0,Aaron Pico,2025-07-26,28.0,68.0,145.0,70.5,NaN,13,5,0,9.0,7.0,2.0,2.0,13.0,29.0,2.0,3.0,4.0,2.0,8.96,0.00,50.0,NaN,0.00,03:21,7.0,3.0,3.0,7.0,5.0,1.0
1,Aaron Jeffery,2025-08-24,30.0,74.0,185.5,73.5,41.0,6,2,0,NaN,NaN,0.0,0.0,57.0,125.0,NaN,4.0,3.0,4.0,0.00,0.00,34.0,50.0,0.00,09:12,23.0,34.0,0.0,30.0,18.0,9.0
2,AJ Cunningham,2024-03-02,30.0,70.0,136.0,71.0,41.0,11,5,0,4.0,5.0,3.0,4.0,121.0,412.0,NaN,3.0,4.0,6.0,0.00,0.52,48.0,46.0,0.00,09:34,118.0,3.0,0.0,65.0,35.0,21.0
3,Aaron Brink,2000-11-17,48.0,75.0,231.0,NaN,NaN,8,3,0,NaN,NaN,0.0,0.0,11.0,26.0,NaN,3.0,3.0,6.0,0.00,0.00,57.0,NaN,0.00,01:51,2.0,9.0,0.0,10.0,1.0,0.0
4,Aalon Cruz,2019-07-30,33.0,72.0,155.0,78.0,42.0,8,4,0,2.0,NaN,4.0,2.0,134.0,343.0,NaN,7.0,8.0,9.0,0.00,0.00,59.0,NaN,0.85,05:54,132.0,1.0,1.0,83.0,36.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,Zarah Fairn,2019-10-05,39.0,68.0,146.0,72.0,42.0,6,5,0,4.0,4.0,2.0,0.0,171.0,543.0,NaN,2.0,5.0,7.0,0.00,0.00,46.0,60.0,0.00,09:24,160.0,11.0,0.0,128.0,26.0,17.0
2996,Zu Anyanwu,2017-07-11,41.0,73.0,263.5,77.0,41.0,14,5,0,NaN,NaN,4.0,2.0,57.0,133.0,NaN,1.0,2.0,2.0,0.65,0.00,69.0,NaN,0.65,11:32,48.0,3.0,6.0,46.0,11.0,0.0
2997,Zygimantas Ramaska,2024-08-24,28.0,71.0,145.5,73.0,NaN,9,3,0,5.0,5.0,0.0,4.0,6.0,10.0,NaN,NaN,1.0,2.0,0.00,2.41,30.0,33.0,0.00,06:14,2.0,4.0,0.0,5.0,1.0,0.0
2998,Zviad Lazishvili,2021-10-23,31.0,66.0,135.0,69.0,37.0,13,1,0,NaN,NaN,4.0,9.0,63.0,160.0,NaN,6.0,4.0,6.0,0.00,0.00,51.0,NaN,0.00,15:00,60.0,3.0,0.0,25.0,17.0,21.0


In [21]:
missing_df = pd.DataFrame({
    'missing_count': df_stats.isnull().sum(),
    'missing_precent': round((df_stats.isnull().sum() / len(df_stats)) * 100, 2)
}).sort_values(by='missing_precent', ascending=False)

missing_df


,missing_count,missing_precent
first_round_finishes,2096,71.22
takedowns_landed,2051,69.69
wins_by_knockout,1761,59.84
leg_reach,1335,45.36
reach,1058,35.95
takedown_defense,665,22.60
takedowns_attempted,522,17.74
sig_strikes_defense,250,8.49
sig_strikes_attempted,247,8.39
sig_strikes_landed,247,8.39


Fill null values with 0 where it makes sense

In [22]:
# For example, it is not unusual for a fighter to have 0 first_round_finishes
columns = ['first_round_finishes', 'takedowns_landed', 'wins_by_knockout', 'takedowns_attempted', 'takedown_defense']

for col in columns:
    df_stats[col] = df_stats[col].fillna(0)

In [23]:
df_stats.isnull().sum()

name                                  0
octagon_debut                         0
age                                 242
height                              239
weight                               72
reach                              1058
leg_reach                          1335
wins                                  0
losses                                0
draws                                 0
wins_by_knockout                      0
first_round_finishes                  0
win_by_dec                            0
win_by_sub                            0
sig_strikes_landed                  247
sig_strikes_attempted               247
takedowns_landed                      0
takedowns_attempted                   0
sig_strikes_landed_per_minute       238
sig_strikes_absorbed_per_minute     238
takedowns_avg                       238
submission_avg                      238
sig_strikes_defense                 250
takedown_defense                      0
knockdown_avg                       238


Impute Missing columns

Weight and Height have some 0 values, this does not make sense as someone can not have a value of height be equal to 0, which will impact the imputation.
I will convert to null and after impute.

In [26]:
df_stats[df_stats['height'] == 0]
df_stats[df_stats['weight'] == 0]

,name,octagon_debut,age,height,weight,reach,leg_reach,wins,losses,draws,wins_by_knockout,first_round_finishes,win_by_dec,win_by_sub,sig_strikes_landed,sig_strikes_attempted,takedowns_landed,takedowns_attempted,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,sig_strikes_defense,takedown_defense,knockdown_avg,fight_time_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
36,Adrian Serrano,2000-06-09,59.0,0.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,00:00,0.0,0.0,0.0,0.0,0.0,0.0
48,Adriano Santos,1998-10-16,NaN,0.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,00:00,0.0,0.0,0.0,0.0,0.0,0.0
58,Alberto Cerro Leon,1994-03-11,NaN,75.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,00:00,0.0,0.0,0.0,0.0,0.0,0.0
76,Alex Andrade,2000-06-09,49.0,0.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,4.0,11.0,0.0,0.0,0.0,3.0,0.00,0.75,53.0,25.0,0.0,20:00,3.0,1.0,0.0,3.0,1.0,0.0
83,Alex Hunter,1997-07-27,NaN,0.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,00:00,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2853,Valeri Ignatov,1999-03-05,NaN,70.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,00:00,0.0,0.0,0.0,0.0,0.0,0.0
2900,Wallid Ismail,1997-02-07,55.0,70.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,30.0,96.0,0.0,14.0,1.0,1.0,1.85,0.46,45.0,0.0,0.0,16:15,1.0,0.0,29.0,27.0,3.0,0.0
2912,Wes Albritton,1997-05-30,NaN,0.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,00:00,0.0,0.0,0.0,0.0,0.0,0.0
2963,Yoshiki Takahashi,1997-02-07,54.0,0.0,0.0,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,15.0,29.0,0.0,1.0,2.0,6.0,0.00,2.26,45.0,67.0,0.0,02:13,6.0,0.0,9.0,11.0,1.0,3.0


In [27]:
df_stats[['height', 'weight']] = df_stats[['height', 'weight']].replace(0, np.nan)

In [29]:
print(df_stats.columns[df_stats.isnull().any()])
print("\n", len(df_stats.columns[df_stats.isnull().any()]))

Index(['age', 'height', 'weight', 'reach', 'leg_reach', 'sig_strikes_landed',
       'sig_strikes_attempted', 'sig_strikes_landed_per_minute',
       'sig_strikes_absorbed_per_minute', 'takedowns_avg', 'submission_avg',
       'sig_strikes_defense', 'knockdown_avg'],
      dtype='object')

 13


Impute all null values

In [30]:
missing_cols = df_stats.columns[df_stats.isnull().any()]

imputer = KNNImputer(n_neighbors=5)
df_stats[missing_cols] = imputer.fit_transform(df_stats[missing_cols])

In [31]:
df_stats.isnull().sum()

name                               0
octagon_debut                      0
age                                0
height                             0
weight                             0
reach                              0
leg_reach                          0
wins                               0
losses                             0
draws                              0
wins_by_knockout                   0
first_round_finishes               0
win_by_dec                         0
win_by_sub                         0
sig_strikes_landed                 0
sig_strikes_attempted              0
takedowns_landed                   0
takedowns_attempted                0
sig_strikes_landed_per_minute      0
sig_strikes_absorbed_per_minute    0
takedowns_avg                      0
submission_avg                     0
sig_strikes_defense                0
takedown_defense                   0
knockdown_avg                      0
fight_time_avg                     0
sig_strikes_standing               0
s

In [32]:
df_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2943 entries, 0 to 2999
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   name                             2943 non-null   object        
 1   octagon_debut                    2943 non-null   datetime64[ns]
 2   age                              2943 non-null   float64       
 3   height                           2943 non-null   float64       
 4   weight                           2943 non-null   float64       
 5   reach                            2943 non-null   float64       
 6   leg_reach                        2943 non-null   float64       
 7   wins                             2943 non-null   int64         
 8   losses                           2943 non-null   int64         
 9   draws                            2943 non-null   int64         
 10  wins_by_knockout                 2943 non-null   float64       
 

In [33]:
df_stats.columns

Index(['name', 'octagon_debut', 'age', 'height', 'weight', 'reach',
       'leg_reach', 'wins', 'losses', 'draws', 'wins_by_knockout',
       'first_round_finishes', 'win_by_dec', 'win_by_sub',
       'sig_strikes_landed', 'sig_strikes_attempted', 'takedowns_landed',
       'takedowns_attempted', 'sig_strikes_landed_per_minute',
       'sig_strikes_absorbed_per_minute', 'takedowns_avg', 'submission_avg',
       'sig_strikes_defense', 'takedown_defense', 'knockdown_avg',
       'fight_time_avg', 'sig_strikes_standing', 'sig_strikes_clinch',
       'sig_strikes_ground', 'head_target', 'body_target', 'leg_target'],
      dtype='object')

Change floats back to int at appropriate columns

In [34]:
cols_to_fix = ['age', 'wins', 'losses', 'draws', 'wins_by_knockout', 'first_round_finishes', 'win_by_dec',
            'win_by_sub', 'sig_strikes_landed', 'sig_strikes_attempted', 'takedowns_landed', 'takedowns_attempted',
            'sig_strikes_landed_per_minute', 'sig_strikes_absorbed_per_minute', 'sig_strikes_standing', 'sig_strikes_clinch', 'sig_strikes_ground',
            'head_target', 'body_target', 'leg_target']

for col in cols_to_fix:
    df_stats[col] = df_stats[col].round().astype('Int64')

Remove Fighters with octagon_debut before 2007

The reason for this is because the sport has changed significantly since then and does not represent the same type of stats we expect from today's fihgters.  <br>
Rules, fighters, fighting styles, techniques have all evolved.  <br>
Why 2007?  <br>
The simple answer is there are a handful of fighters that are still active and have been fighting since this year onwards.

In [35]:
df_stats = df_stats[df_stats['octagon_debut'] >= '2007-01-01']

In [36]:
df_stats = df_stats.reset_index(drop=True)

df_stats

,name,octagon_debut,age,height,weight,reach,leg_reach,wins,losses,draws,wins_by_knockout,first_round_finishes,win_by_dec,win_by_sub,sig_strikes_landed,sig_strikes_attempted,takedowns_landed,takedowns_attempted,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,sig_strikes_defense,takedown_defense,knockdown_avg,fight_time_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
0,Aaron Pico,2025-07-26,28,68.0,145.0,70.5,38.0,13,5,0,9,7,2,2,13,29,2,3,4,2,8.96,0.00,50.0,0.0,0.00,03:21,7,3,3,7,5,1
1,Aaron Jeffery,2025-08-24,30,74.0,185.5,73.5,41.0,6,2,0,0,0,0,0,57,125,0,4,3,4,0.00,0.00,34.0,50.0,0.00,09:12,23,34,0,30,18,9
2,AJ Cunningham,2024-03-02,30,70.0,136.0,71.0,41.0,11,5,0,4,5,3,4,121,412,0,3,4,6,0.00,0.52,48.0,46.0,0.00,09:34,118,3,0,65,35,21
3,Aalon Cruz,2019-07-30,33,72.0,155.0,78.0,42.0,8,4,0,2,0,4,2,134,343,0,7,8,9,0.00,0.00,59.0,0.0,0.85,05:54,132,1,1,83,36,15
4,AJ Fletcher,2022-03-12,26,70.0,183.0,67.0,37.0,10,3,0,4,6,1,4,163,329,4,14,3,5,1.54,0.93,47.0,33.0,0.31,09:43,138,12,13,113,18,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480,Zarah Fairn,2019-10-05,39,68.0,146.0,72.0,42.0,6,5,0,4,4,2,0,171,543,0,2,5,7,0.00,0.00,46.0,60.0,0.00,09:24,160,11,0,128,26,17
2481,Zu Anyanwu,2017-07-11,41,73.0,263.5,77.0,41.0,14,5,0,0,0,4,2,57,133,0,1,2,2,0.65,0.00,69.0,0.0,0.65,11:32,48,3,6,46,11,0
2482,Zygimantas Ramaska,2024-08-24,28,71.0,145.5,73.0,38.8,9,3,0,5,5,0,4,6,10,0,0,1,2,0.00,2.41,30.0,33.0,0.00,06:14,2,4,0,5,1,0
2483,Zviad Lazishvili,2021-10-23,31,66.0,135.0,69.0,37.0,13,1,0,0,0,4,9,63,160,0,6,4,6,0.00,0.00,51.0,0.0,0.00,15:00,60,3,0,25,17,21


Round float columns

In [37]:
cols_to_fix = ['height', 'weight', 'reach', 'leg_reach', 'knockdown_avg']

for col in cols_to_fix:
    df_stats[col] = df_stats[col].round(2)

Concat df_stats & df_events

In [38]:
df_events.head()

,id,event_date,event_name,fighter_blue,fighter_red,knockdowns_blue,knockdowns_red,sig_attempts_blue,sig_attempts_red,sig_strikes_blue,sig_strikes_red,total_strikes_attempts_blue,total_strikes_attempts_red,total_strikes_blue,total_strikes_red,sub_attempts_blue,sub_attempts_red,takedowns_blue,takedowns_red,takedown_attempts_blue,takedown_attempts_red,control_time_blue,control_time_red,round,time,weight_class,win_method,winner
0,1,1994-03-11,UFC 2: No Way Out,Scott Morris,Patrick Smith,0,0,0,17,0,13,0,25,0,19,0,0,0,0,1,0,0,0,1,30,Open Weight,KO/TKO,Patrick Smith
1,2,1994-03-11,UFC 2: No Way Out,Ray Wizard,Patrick Smith,0,0,1,1,1,1,2,1,2,1,0,1,0,0,0,1,0,0,1,58,Open Weight,Submission,Patrick Smith
2,3,1994-03-11,UFC 2: No Way Out,Robert Lucarelli,Orlando Wiet,0,0,6,12,2,8,6,15,2,11,1,0,1,0,1,0,0,0,1,170,Open Weight,KO/TKO,Orlando Wiet
3,4,1994-03-11,UFC 2: No Way Out,Alberta Cerra Leon,Remco Pardoel,0,0,3,6,1,4,11,22,9,20,0,1,0,1,0,1,0,0,1,591,Open Weight,Submission,Remco Pardoel
4,5,1994-03-11,UFC 2: No Way Out,Thaddeus Luster,Frank Hamaker,0,0,0,3,0,2,0,15,0,14,0,3,0,1,1,1,0,0,1,292,Open Weight,Submission,Frank Hamaker


Drop stat columns from each event

In [39]:
df_events.drop(columns=['id', 'knockdowns_blue', 'knockdowns_red', 'sig_attempts_blue', 'sig_attempts_red', 'sig_strikes_blue', 'sig_strikes_red', 'total_strikes_attempts_blue', 'total_strikes_attempts_red', 
'total_strikes_blue', 'total_strikes_red', 'sub_attempts_blue', 'sub_attempts_red', 'takedowns_blue', 'takedowns_red', 'takedown_attempts_blue', 'takedown_attempts_red', 'control_time_blue', 'control_time_red',
'round', 'time'], inplace=True)
df_events

,event_date,event_name,fighter_blue,fighter_red,weight_class,win_method,winner
0,1994-03-11,UFC 2: No Way Out,Scott Morris,Patrick Smith,Open Weight,KO/TKO,Patrick Smith
1,1994-03-11,UFC 2: No Way Out,Ray Wizard,Patrick Smith,Open Weight,Submission,Patrick Smith
2,1994-03-11,UFC 2: No Way Out,Robert Lucarelli,Orlando Wiet,Open Weight,KO/TKO,Orlando Wiet
3,1994-03-11,UFC 2: No Way Out,Alberta Cerra Leon,Remco Pardoel,Open Weight,Submission,Remco Pardoel
4,1994-03-11,UFC 2: No Way Out,Thaddeus Luster,Frank Hamaker,Open Weight,Submission,Frank Hamaker
...,...,...,...,...,...,...,...
8298,2025-08-23,UFC Fight Night: Walker vs. Zhang,Kevin Borjas,Sumudaerji,Flyweight,Decision - Unanimous,Sumudaerji
8299,2025-08-23,UFC Fight Night: Walker vs. Zhang,Zhang Mingyang,Johnny Walker,Light Heavyweight,KO/TKO,Johnny Walker
8300,2025-08-23,UFC Fight Night: Walker vs. Zhang,Aljamain Sterling,Brian Ortega,Catch Weight,Decision - Unanimous,Aljamain Sterling
8301,2025-08-23,UFC Fight Night: Walker vs. Zhang,Diyar Nurgozhay,Uran Satybaldiev,Light Heavyweight,Submission,Uran Satybaldiev


In [40]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8303 entries, 0 to 8302
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   event_date    8303 non-null   object
 1   event_name    8303 non-null   object
 2   fighter_blue  8303 non-null   object
 3   fighter_red   8303 non-null   object
 4   weight_class  8303 non-null   object
 5   win_method    8303 non-null   object
 6   winner        8156 non-null   object
dtypes: object(7)
memory usage: 454.2+ KB


Keep dates equal and larger than the octagon_debut

In [41]:
df_events['event_date'] = pd.to_datetime(df_events['event_date'], errors='coerce')

min_date = df_stats['octagon_debut'].min()
df_events = df_events[df_events['event_date'] >= min_date]
df_events.reset_index(drop=True, inplace=True)

In [42]:
df_events.columns

Index(['event_date', 'event_name', 'fighter_blue', 'fighter_red',
       'weight_class', 'win_method', 'winner'],
      dtype='object')

In [43]:
df_stats.columns

Index(['name', 'octagon_debut', 'age', 'height', 'weight', 'reach',
       'leg_reach', 'wins', 'losses', 'draws', 'wins_by_knockout',
       'first_round_finishes', 'win_by_dec', 'win_by_sub',
       'sig_strikes_landed', 'sig_strikes_attempted', 'takedowns_landed',
       'takedowns_attempted', 'sig_strikes_landed_per_minute',
       'sig_strikes_absorbed_per_minute', 'takedowns_avg', 'submission_avg',
       'sig_strikes_defense', 'takedown_defense', 'knockdown_avg',
       'fight_time_avg', 'sig_strikes_standing', 'sig_strikes_clinch',
       'sig_strikes_ground', 'head_target', 'body_target', 'leg_target'],
      dtype='object')

Left Join events by both fighter stats of each event fight

In [44]:
# First, ensure the fighter names are clean for merging
df_events.loc[:, 'fighter_blue'] = df_events['fighter_blue'].str.strip()
df_events.loc[:, 'fighter_red'] = df_events['fighter_red'].str.strip()
df_stats.loc[:, 'name'] = df_stats['name'].str.strip()

# Merge stats for fighter_blue
df_merged = df_events.merge(
    df_stats,
    how='left',
    left_on='fighter_blue',
    right_on='name',
    suffixes=('', '_drop') 
)

# Rename fighter_blue stats columns with _blue suffix (excluding the original 'name')
cols_to_rename_blue = [col for col in df_stats.columns if col != 'name']
df_merged.rename(columns={col: f"{col}_blue" for col in cols_to_rename_blue}, inplace=True)
df_merged.drop(columns='name', inplace=True)

# Merge stats for fighter_red
df_merged = df_merged.merge(
    df_stats,
    how='left',
    left_on='fighter_red',
    right_on='name',
    suffixes=('', '_drop') 
)

# Rename fighter_red stats columns with _red suffix
cols_to_rename_red = [col for col in df_stats.columns if col != 'name']
df_merged.rename(columns={col: f"{col}_red" for col in cols_to_rename_red}, inplace=True)
df_merged.drop(columns='name', inplace=True)

In [45]:
df_merged

,event_date,event_name,fighter_blue,fighter_red,weight_class,win_method,winner,octagon_debut_blue,age_blue,height_blue,weight_blue,reach_blue,leg_reach_blue,wins_blue,losses_blue,draws_blue,wins_by_knockout_blue,first_round_finishes_blue,win_by_dec_blue,win_by_sub_blue,sig_strikes_landed_blue,sig_strikes_attempted_blue,takedowns_landed_blue,takedowns_attempted_blue,sig_strikes_landed_per_minute_blue,sig_strikes_absorbed_per_minute_blue,takedowns_avg_blue,submission_avg_blue,sig_strikes_defense_blue,takedown_defense_blue,knockdown_avg_blue,fight_time_avg_blue,sig_strikes_standing_blue,sig_strikes_clinch_blue,sig_strikes_ground_blue,head_target_blue,body_target_blue,leg_target_blue,octagon_debut_red,age_red,height_red,weight_red,reach_red,leg_reach_red,wins_red,losses_red,draws_red,wins_by_knockout_red,first_round_finishes_red,win_by_dec_red,win_by_sub_red,sig_strikes_landed_red,sig_strikes_attempted_red,takedowns_landed_red,takedowns_attempted_red,sig_strikes_landed_per_minute_red,sig_strikes_absorbed_per_minute_red,takedowns_avg_red,submission_avg_red,sig_strikes_defense_red,takedown_defense_red,knockdown_avg_red,fight_time_avg_red,sig_strikes_standing_red,sig_strikes_clinch_red,sig_strikes_ground_red,head_target_red,body_target_red,leg_target_red
0,2007-01-25,UFC Fight Night: Evans vs Salmon,Heath Herring,Jake O'Brien,Heavyweight,Decision - Unanimous,Jake O'Brien,2007-01-25,45,76.0,250.0,77.7,41.8,28,14,0,0,0,0,0,583,1157,0,40,2,2,1.22,0.73,55.0,33.0,0.37,12:19,220,66,297,302,158,123,NaT,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2007-01-25,UFC Fight Night: Evans vs Salmon,Clay Guida,Din Thomas,Lightweight,Decision - Unanimous,Din Thomas,NaT,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2007-01-25,UFC Fight Night: Evans vs Salmon,Chris Price,Ed Herman,Middleweight,Submission,Ed Herman,NaT,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,2007-01-25,UFC Fight Night: Evans vs Salmon,Ross Pointon,Rich Clementi,Welterweight,Submission,Rich Clementi,NaT,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,2007-01-25,UFC Fight Night: Evans vs Salmon,Dean Lister,Nate Marquardt,Middleweight,Decision - Unanimous,Nate Marquardt,NaT,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7641,2025-08-23,UFC Fight Night: Walker vs. Zhang,Kevin Borjas,Sumudaerji,Flyweight,Decision - Unanimous,Sumudaerji,2023-11-11,27,65.0,125.0,68.0,36.0,10,4,0,8,5,2,0,263,588,0,5,5,6,0.00,0.00,51.0,65.0,0.29,12:54,253,6,4,214,30,19,2025-08-24,29,68.0,126.0,72.0,38.5,18,7,0,13,11,4,1,358,698,0,4,4,3,0.18,0.74,60.0,68.0,0.37,10:10,279,29,50,227,44,87
7642,2025-08-23,UFC Fight Night: Walker vs. Zhang,Zhang Mingyang,Johnny Walker,Light Heavyweight,KO/TKO,Johnny Walker

In [46]:
df_merged.isnull().sum()

event_date                   0
event_name                   0
fighter_blue                 0
fighter_red                  0
weight_class                 0
                          ... 
sig_strikes_clinch_red    1129
sig_strikes_ground_red    1129
head_target_red           1129
body_target_red           1129
leg_target_red            1129
Length: 69, dtype: int64

Drop rows where fighters have null stats, these rows are from fighters that have not been found during the join and have been assigned a null value

In [47]:
df_merged.columns

Index(['event_date', 'event_name', 'fighter_blue', 'fighter_red',
       'weight_class', 'win_method', 'winner', 'octagon_debut_blue',
       'age_blue', 'height_blue', 'weight_blue', 'reach_blue',
       'leg_reach_blue', 'wins_blue', 'losses_blue', 'draws_blue',
       'wins_by_knockout_blue', 'first_round_finishes_blue', 'win_by_dec_blue',
       'win_by_sub_blue', 'sig_strikes_landed_blue',
       'sig_strikes_attempted_blue', 'takedowns_landed_blue',
       'takedowns_attempted_blue', 'sig_strikes_landed_per_minute_blue',
       'sig_strikes_absorbed_per_minute_blue', 'takedowns_avg_blue',
       'submission_avg_blue', 'sig_strikes_defense_blue',
       'takedown_defense_blue', 'knockdown_avg_blue', 'fight_time_avg_blue',
       'sig_strikes_standing_blue', 'sig_strikes_clinch_blue',
       'sig_strikes_ground_blue', 'head_target_blue', 'body_target_blue',
       'leg_target_blue', 'octagon_debut_red', 'age_red', 'height_red',
       'weight_red', 'reach_red', 'leg_reach_red', 'wi

In [48]:
df_merged = df_merged.dropna(subset=['octagon_debut_blue',
    'age_blue', 'height_blue', 'weight_blue', 'reach_blue',
    'leg_reach_blue', 'wins_blue', 'losses_blue', 'draws_blue',
    'wins_by_knockout_blue', 'first_round_finishes_blue', 'win_by_dec_blue',
    'win_by_sub_blue', 'sig_strikes_landed_blue',
    'sig_strikes_attempted_blue', 'takedowns_landed_blue',
    'takedowns_attempted_blue', 'sig_strikes_landed_per_minute_blue',
    'sig_strikes_absorbed_per_minute_blue', 'takedowns_avg_blue',
    'submission_avg_blue', 'sig_strikes_defense_blue',
    'takedown_defense_blue', 'knockdown_avg_blue', 'fight_time_avg_blue',
    'sig_strikes_standing_blue', 'sig_strikes_clinch_blue',
    'sig_strikes_ground_blue', 'head_target_blue', 'body_target_blue',
    'leg_target_blue', 'octagon_debut_red',
    'age_red', 'height_red', 'weight_red', 'reach_red', 'leg_reach_red',
    'wins_red', 'losses_red', 'draws_red', 'wins_by_knockout_red',
    'first_round_finishes_red', 'win_by_dec_red', 'win_by_sub_red',
    'sig_strikes_landed_red', 'sig_strikes_attempted_red',
    'takedowns_landed_red', 'takedowns_attempted_red',
    'sig_strikes_landed_per_minute_red',
    'sig_strikes_absorbed_per_minute_red', 'takedowns_avg_red',
    'submission_avg_red', 'sig_strikes_defense_red', 'takedown_defense_red',
    'knockdown_avg_red', 'fight_time_avg_red', 'sig_strikes_standing_red',
    'sig_strikes_clinch_red', 'sig_strikes_ground_red', 'head_target_red',
    'body_target_red', 'leg_target_red'])


In [49]:
df_merged = df_merged.sort_values(by=['event_date']).reset_index(drop=True)

In [50]:
df_merged

,event_date,event_name,fighter_blue,fighter_red,weight_class,win_method,winner,octagon_debut_blue,age_blue,height_blue,weight_blue,reach_blue,leg_reach_blue,wins_blue,losses_blue,draws_blue,wins_by_knockout_blue,first_round_finishes_blue,win_by_dec_blue,win_by_sub_blue,sig_strikes_landed_blue,sig_strikes_attempted_blue,takedowns_landed_blue,takedowns_attempted_blue,sig_strikes_landed_per_minute_blue,sig_strikes_absorbed_per_minute_blue,takedowns_avg_blue,submission_avg_blue,sig_strikes_defense_blue,takedown_defense_blue,knockdown_avg_blue,fight_time_avg_blue,sig_strikes_standing_blue,sig_strikes_clinch_blue,sig_strikes_ground_blue,head_target_blue,body_target_blue,leg_target_blue,octagon_debut_red,age_red,height_red,weight_red,reach_red,leg_reach_red,wins_red,losses_red,draws_red,wins_by_knockout_red,first_round_finishes_red,win_by_dec_red,win_by_sub_red,sig_strikes_landed_red,sig_strikes_attempted_red,takedowns_landed_red,takedowns_attempted_red,sig_strikes_landed_per_minute_red,sig_strikes_absorbed_per_minute_red,takedowns_avg_red,submission_avg_red,sig_strikes_defense_red,takedown_defense_red,knockdown_avg_red,fight_time_avg_red,sig_strikes_standing_red,sig_strikes_clinch_red,sig_strikes_ground_red,head_target_red,body_target_red,leg_target_red
0,2007-04-05,UFC Fight Night: Stevenson vs Guillard,Naoyuki Kotani,Thiago Tavares,Lightweight,Decision - Unanimous,Thiago Tavares,2007-04-05,37,68.0,156.0,72.1,39.8,33,13,7,0,0,3,25,55,177,0,18,1,3,0.83,0.83,50.0,47.0,0.00,09:00,40,4,11,30,4,21,2007-04-05,38,67.0,146.0,68.0,39.0,24,7,1,0,0,0,0,329,820,0,98,2,2,3.83,1.46,63.0,63.0,0.00,09:08,209,30,90,229,76,24
1,2007-04-21,UFC 70: Nations Collide,Dennis Siver,Jess Liaudin,Welterweight,Submission,Jess Liaudin,2007-04-21,44,65.0,146.0,70.0,38.0,23,11,0,0,0,0,0,869,2678,13,40,4,3,0.87,0.33,65.0,65.0,0.47,10:41,737,46,86,494,118,257,2007-04-21,32,69.0,155.0,71.3,39.2,12,11,0,0,0,0,0,90,175,0,15,2,3,3.28,1.64,42.0,31.0,0.82,07:19,40,35,15,70,9,11
2,2007-04-21,UFC 70: Nations Collide,Matt Grice,Terry Etim,Lightweight,Submission,Terry Etim,2007-04-21,41,68.0,145.0,70.0,37.9,15,5,0,0,0,0,0,252,472,0,24,4,3,2.19,0.49,56.0,82.0,0.73,08:48,68,59,125,196,33,23,2007-04-21,37,73.0,155.0,73.0,39.7,15,5,0,0,0,0,0,229,639,0,10,2,3,0.43,1.42,67.0,39.0,0.28,09:37,199,27,3,110,25,94
3,2007-06-12,UFC Fight Night: Stout vs Fisher,Jason Black,Thiago Tavares,Lightweight,Submission,Thiago Tavares,2007-06-12,50,68.0,156.0,72.9,40.8,22,4,1,0,0,0,0,33,69,0,3,1,3,0.48,6.25,45.0,29.0,0.00,09:22,9,7,17,27,6,0,2007-04-05,38,67.0,146.0,68.0,39.0,24,7,1,0,0,0,0,329,820,0,98,2,2,3.83,1.46,63.0,63.0,0.00,09:08,209,30,90,229,76,24
4,2007-06-12,UFC Fight Night: Stout vs Fisher,Chad Reiner,Anthony Johnson,Welterweight,KO/TKO,Anthony Johnson,2007-01-25,41,71.0,171.0,72.4,40.2,13,2,0,0,0,0,0,8,39,0,3,1,2,0.99,1.97,46.0,25.0,0.00,07:37,5,3,0,4,2,2,2007-06-12,39,74.0,203.8,78.0,44.0,22,6,0,0,0,0,0,381,800,0,33,3,2,2.43,0.64,60.0,77.0,1.66,06:10,214,67,100,255,57,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5864,2025-08-23,UFC Fight Night: Walker vs. Zhang,Kevin Borjas,Sumudaerji,Flyweight,Decision - Unanimous,Sumudaerji,2023-11-11,27,65.0,125.0,68.0,36.0,10,4,0,8,5,2,0,263,588,0,5,5,6,0.00,0.00,51.0,65.0,0.29,12:54,253,6,4,214,30,19,2025-08-24,29,68.0,126.0,72.0,38.5,18,7,0,13,11,4,1,358,698,0,4,4,3,0.18,0.74,60.0,68.0,0.37,10:10,279,29,50,227,44,87
5865,2025-08-23,UFC Fight Night: Walker vs. Zhang,Aljamain Sterling,Brian Ortega,Catch Weight,Decision - Unanimous,Aljamain Sterling,2014-02-22,36,67.0,145.5,71.0,39.0,25,5,0,3,4,14,8,1237,2359,21,157,4,2,2.51,0.69,59.0,43.0,0.00,13:22,816,187,234,601,410,226,2014-07-26,34,68.0,146.0,69.0,39.0,16,5,0,3,4,5,8,687,1813,1,46,4,7,1.07,1.07,50.0,56.0,0.45,12:57,603,49,35,498,116,73
5866,2025-08-23,UFC 

Save to csv for further use

In [51]:
df_merged.to_csv("../data/merged_df.csv", index=False)